In [2]:
require(adelie)
require(glmnet)
require(grplasso)
require(SGL)
require(sparsegl)
require(gglasso)
require(grpreg)
require(grpnet)

In [5]:
objective <- function(
    beta, X, y, groups, penalty, lmda, family="gaussian"
)
{
    group_sizes <- c(groups, ncol(X))
    group_sizes <- group_sizes[2:(length(groups)+1)] - group_sizes[1:length(groups)]
    eta <- as.numeric(X %*% beta)
    if (family == "gaussian") {
        r <- y - eta
        obj <- 0.5 * mean(r ** 2)
    } else {
        obj <- mean(-eta * y + log(1 + exp(eta)))
    }

    for (i in 1:length(groups)) {
        g <- groups[i]
        gs <- group_sizes[i]
        omega <- penalty[i]
        obj <- obj + lmda * omega * sqrt(sum(beta[(1+g) : (g+gs)] ** 2))
    }
    obj
}

In [6]:
benchmark <- function(
    X,
    y,
    G,
    lmdas, # ignored if pkg=="adelie"
    pkg,
    n_sims,
    family="gaussian"
)
{
    n <- nrow(X)
    p <- ncol(X)
    penalty.factor <- rep(sqrt(p / G), G)
    total_time <- 0

    if (pkg == "adelie") {
        if (family == "gaussian") {
            glm = adelie::glm.gaussian(y)
        } else {
            glm = adelie::glm.binomial(y)
        }
        for (i in 1:n_sims) {
            start <- proc.time()
            out <- adelie::grpnet(
                X=X,
                glm=glm,
                groups=(p / G) * ((1:G) - 1),
                alpha=1,
                penalty=penalty.factor,
                early_exit=FALSE,
                min_ratio=1e-2,
                lmda_path_size=100,
                intercept=FALSE,
                screen_rule="strong"
            )
            end <- proc.time()
            total_time <- total_time + (end - start)[1]
        }
        stopifnot(length(out$lmdas) == 100) 

    } else if (pkg == "glmnet") {
        stopifnot(G == p)
        for (i in 1:n_sims) {
            start <- proc.time()
            out <- glmnet::glmnet(
                x=X,
                y=y,
                family=family,
                alpha=1,
                lambda=lmdas,
                standardize=FALSE,
                intercept=FALSE,
                type.gaussian="naive",
                penalty.factor=penalty.factor
            )
            end <- proc.time()
            total_time <- total_time + (end - start)[1]
        }
        stopifnot(length(out$lambda) == 100) 

    } else if (pkg == "grplasso") {
        if (family == "gaussian") {
            model = grplasso::LinReg()
        } else {
            model = grplasso::LogReg()
        }
        for (i in 1:n_sims) {
            start <- proc.time()
            out <- grplasso::grplasso(
                x=X,
                y=y,
                index=as.numeric(rep(1:G, each=p/G)),
                lambda=lmdas,
                model=model,
                center=FALSE,
                standardize=FALSE,
                control=grplasso::grpl.control(trace=0),
            )
            end <- proc.time()
            total_time <- total_time + (end - start)[1]
        }
        stopifnot(length(out$lambda) == length(lmdas)) 

    } else if (pkg == "SGL") {
        if (family == "gaussian") {
            type = "linear"
        } else {
            type = "logit"
        }
        for (i in 1:n_sims) {
            start <- proc.time()
            out <- SGL::SGL(
                data=list(x=X, y=y),
                index=as.numeric(rep(1:G, each=p/G)),
                type=type,
                standardize=FALSE,
                alpha=0,
                lambdas=lmdas,
            )
            end <- proc.time()
            total_time <- total_time + (end - start)[1]
        }
        stopifnot(length(out$lambdas) == length(lmdas)) 

    } else if (pkg == "sparsegl") {
        for (i in 1:n_sims) {
            start <- proc.time()
            out <- sparsegl::sparsegl(
                x=X,
                y=y,
                group=as.numeric(rep(1:G, each=p/G)),
                family=family,
                lambda=lmdas,
                pf_group=penalty.factor,
                intercept=FALSE,
                asparse=0,
                standardize=FALSE,
            )
            end <- proc.time()
            total_time <- total_time + (end - start)[1]
        }
        stopifnot(length(out$lambda) == length(lmdas)) 

    } else if (pkg == "gglasso") {
        if (family == "gaussian") {
            loss = "ls"
        } else {
            loss = "logit"
            y <- 2 * y - 1
        }
        for (i in 1:n_sims) {
            start <- proc.time()
            out <- gglasso::gglasso(
                x=X,
                y=y,
                group=as.numeric(rep(1:G, each=p/G)),
                loss=loss,
                lambda=lmdas,
                pf=penalty.factor,
                intercept=FALSE,
            )
            end <- proc.time()
            total_time <- total_time + (end - start)[1]
        }
        stopifnot(length(out$lambda) == length(lmdas)) 

    } else if (pkg == "grpreg") {
        for (i in 1:n_sims) {
            start <- proc.time()
            out <- grpreg::grpreg(
                X=X,
                y=y,
                group=as.numeric(rep(1:G, each=p/G)),
                penalty="grLasso",
                family=family,
                lambda=lmdas,
                alpha=1,
                group.multiplier=penalty.factor,
            )
            end <- proc.time()
            total_time <- total_time + (end - start)[1]
        }
        stopifnot(length(out$lambda) == length(lmdas)) 

    } else if (pkg == "grpnet") {
        for (i in 1:n_sims) {
            start <- proc.time()
            out <- grpnet::grpnet(
                x=X,
                y=y, 
                group=factor(rep(1:G, each=p/G)),
                family=family,
                alpha=1,
                lambda=lmdas,
                penalty.factor=penalty.factor,
                penalty="LASSO",
                standardize=FALSE,
                orthogonalize=FALSE,
                intercept=FALSE,
            )
            end <- proc.time()
            total_time <- total_time + (end - start)[1]
        }
        stopifnot(length(out$lambda) == length(lmdas)) 

    } else {
        stop(paste("Unrecognized package:", pkg))
    }

    list(
        "out"=out,
        "time"=total_time / n_sims
    )
}

In [7]:
generate.data <- function(n, G, rho, p=NULL, snr=3, seed=0, family="gaussian", method="basis")
{
    set.seed(seed)

    if (method == "basis") {
        W <- rnorm(n)
        Z <- matrix(rnorm(n * G), n, G)
        X <- sqrt(1-rho) * Z + sqrt(rho) * W
        X <- sapply(1:G, function(i) cbind(X[,i], X[,i]**2, X[,i]**3))
        X <- matrix(X, n, 3 * G)

        # compute Var(X)    
        var_X <- matrix(c(
            1, 0, 3,
            0, 3, 0,
            3, 0, 15
        ), 3, 3)
        cov_X <- matrix(c(
            rho, 0, 3*rho,
            0, rho**2*3+1-rho**2, 0,
            3 * rho**3 + 3 * rho * (1-rho**2), 0, rho**3*15+9*rho*(1-rho**2)
        ), 3, 3)
        fullvar_X <- as.matrix(rbind(cbind(var_X, cov_X), cbind(cov_X, var_X)))
        
        beta <- rnorm(6)
        signal <- sum(beta * (fullvar_X %*% beta))
        noise <- sqrt(signal / snr)

        eta <- as.numeric(X[,1:6] %*% beta + noise * rnorm(n))
    } else {
        stopifnot(p %% G == 0)
        W <- rnorm(n)
        Z <- matrix(rnorm(n * p), n, p)
        X <- sqrt(1-rho) * Z + sqrt(rho) * W
        gsize <- p / G
        beta <- rnorm(gsize)
        signal <- rho * sum(beta) ** 2 + (1-rho) * sum(beta**2)
        noise <- sqrt(signal / snr)
        eta <- as.numeric(X[,1:gsize] %*% beta + noise * rnorm(n))
    }

    if (family == "gaussian") {
        y <- eta
        # standardize
        y <- y - mean(y)
        y <- y / sqrt(mean(y ** 2))
    } else {
        y <- rbinom(n, 1, 1/(1 + exp(-eta)))
    }

    # standardize
    X <- t(t(X) - colMeans(X))
    X <- t(t(X) / sqrt(colMeans(X ** 2)))

    list(X=X, y=y)
}

In [8]:
run_benchmarks <- function(
    configs,
    generate.data.func,
    snr=3,
    seed=0,
    family="gaussian",
    n_times=3,
    obj_tol=1e-5
)
{
    # n, G, rho, packages
    colnames_ <- c(
        "n", "G", "rho", 
        "adelie", 
        "glmnet",
        #"grplasso", 
        #"SGL", 
        "sparsegl", 
        "gglasso", 
        #"grpreg",
        "grpnet"
    )
    times <- matrix(NA, nrow(configs), length(colnames_)) 
    objectives <- matrix(NA, nrow(configs), length(colnames_))
    colnames(times) <- colnames_
    colnames(objectives) <- colnames_

    for (i in 1:nrow(configs)) {
        n <- configs[i,1]
        G <- configs[i,2]
        rho <- configs[i,3]
        
        data <- generate.data.func(n, G, rho, snr=snr, seed=seed, family=family)
        X <- data$X
        y <- data$y
        group_size <- ncol(X) / G
        penalty.factor <- rep(sqrt(group_size), G)
        groups <- group_size * ((1:G) - 1)
        obj_idx <- 100

        bench.adelie <- benchmark(X, y, G, NA, "adelie", n_times, family=family)
        obj.adelie <- objective(bench.adelie$out$betas[obj_idx,], X, y, groups, penalty.factor, bench.adelie$out$lmdas[obj_idx], family=family)

        if (group_size == 1) {
            bench.glmnet <- benchmark(X, y, G, bench.adelie$out$lmdas, "glmnet", n_times, family=family)
            obj.glmnet <- objective(bench.glmnet$out$beta[,obj_idx], X, y, groups, penalty.factor, bench.adelie$out$lmdas[obj_idx], family=family)
        } else {
            bench.glmnet <- list()
            bench.glmnet$time <- NA
            obj.glmnet <- NA
        }

        #bench.grplasso <- benchmark(X, y, G, 2 * bench.adelie$out$lmdas * n, "grplasso", n_times, family=family)
        #obj.grplasso <- objective(bench.grplasso$out$coefficients[,obj_idx], X, y, groups, penalty.factor, bench.adelie$out$lmdas[obj_idx], family=family)
        #if (obj.grplasso > obj.adelie + obj_tol) {
        #    warning(paste("n=", n, ", G=", G, ", rho=", rho, " pkg=", "grplasso", sep=""))
        #}

        # Doesn't converge!
        #bench.sgl <- benchmark(X, y, G, bench.adelie$out$lmdas, "SGL", n_times, family=family)
        #obj.sgl <- objective(bench.sgl$out$beta[,obj_idx], X, y, groups, penalty.factor, bench.adelie$out$lmdas[obj_idx], family=family)
        #if (obj.sgl > obj.adelie + obj_tol) {
        #    warning(paste("n=", n, ", G=", G, ", rho=", rho, " pkg=", "SGL", sep=""))
        #}

        bench.sparsegl <- benchmark(X, y, G, bench.adelie$out$lmdas, "sparsegl", n_times, family=family)
        obj.sparsegl <- objective(bench.sparsegl$out$beta[,obj_idx], X, y, groups, penalty.factor, bench.adelie$out$lmdas[obj_idx], family=family)
        if (obj.sparsegl > obj.adelie + obj_tol) {
            warning(paste("n=", n, ", G=", G, ", rho=", rho, " pkg=", "sparsegl", sep=""))
        }

        bench.gglasso <- benchmark(X, y, G, bench.adelie$out$lmdas, "gglasso", n_times, family=family)
        obj.gglasso <- objective(bench.gglasso$out$beta[,obj_idx], X, y, groups, penalty.factor, bench.adelie$out$lmdas[obj_idx], family=family)
        if (obj.gglasso > obj.adelie + obj_tol) {
            warning(paste("n=", n, ", G=", G, ", rho=", rho, " pkg=", "gglasso", sep=""))
        }

        #bench.grpreg <- benchmark(X, y, G, bench.adelie$out$lmdas / n, "grpreg", n_times, family=family)
        #obj.grpreg <- objective(bench.grpreg$out$beta[2:(ncol(X)+1),obj_idx], X, y, groups, penalty.factor, bench.adelie$out$lmdas[obj_idx], family=family)
        #if (obj.grpreg > obj.adelie + obj_tol) {
        #    warning(paste("n=", n, ", G=", G, ", rho=", rho, " pkg=", "grpreg", sep=""))
        #}

        bench.grpnet <- benchmark(X, y, G, bench.adelie$out$lmdas, "grpnet", n_times, family=family)
        obj.grpnet <- objective(bench.grpnet$out$beta[,obj_idx], X, y, groups, penalty.factor, bench.adelie$out$lmdas[obj_idx], family=family)
        if (obj.grpnet > obj.adelie + obj_tol) {
            warning(paste("n=", n, ", G=", G, ", rho=", rho, " pkg=", "grpnet", sep=""))
        }
        
        times[i,] <- c(
            n, G, rho, 
            bench.adelie$time, 
            bench.glmnet$time, 
            #bench.grplasso$time, 
            #bench.sgl$time, 
            bench.sparsegl$time, 
            bench.gglasso$time,
            #bench.grpreg$time,
            bench.grpnet$time
        )
        objectives[i,] <- c(
            n, G, rho,
            obj.adelie,
            obj.glmnet,
            obj.sparsegl,
            obj.gglasso,
            obj.grpnet
        )
        curr_row <- matrix(times[i,], nrow=1)
        colnames(curr_row) <- colnames_
        print(curr_row)
        curr_row <- matrix(objectives[i,], nrow=1)
        colnames(curr_row) <- colnames_
        print(curr_row)
        flush.console()
    }

    list(times=times, objectives=objectives)
}

In [9]:
configs <- t(matrix(c(
    # n, G, rho
    100, 100, 0.0,
    100, 1000, 0.0,
    100, 5000, 0.0,
    100, 10000, 0.0,
    100, 50000, 0.0,
    100, 100000, 0.0,

    100, 100, 0.5,
    100, 1000, 0.5,
    100, 5000, 0.5,
    100, 10000, 0.5,
    100, 50000, 0.5,
    100, 100000, 0.5,

    100, 100, 0.95,
    100, 1000, 0.95,
    100, 5000, 0.95,
    100, 10000, 0.95,
    100, 50000, 0.95,
    100, 100000, 0.95,

    1000, 10, 0.0,
    1000, 100, 0.0,
    1000, 500, 0.0,
    1000, 1000, 0.0,
    1000, 5000, 0.0,
    1000, 10000, 0.0,

    1000, 10, 0.5,
    1000, 100, 0.5,
    1000, 500, 0.5,
    1000, 1000, 0.5,
    1000, 5000, 0.5,
    1000, 10000, 0.5,

    1000, 10, 0.95,
    1000, 100, 0.95,
    1000, 500, 0.95,
    1000, 1000, 0.95,
    1000, 5000, 0.95,
    1000, 10000, 0.95
), nrow=3))

In [44]:
out.gaussian <- run_benchmarks(configs, generate.data, obj_tol=1e-4, family="gaussian")

       n   G rho     adelie glmnet   sparsegl    gglasso     grpnet
[1,] 100 100   0 0.01166667     NA 0.04433333 0.04766667 0.01766667
       n   G rho     adelie glmnet   sparsegl    gglasso     grpnet
[1,] 100 100   0 0.04127786     NA 0.04127519 0.04127517 0.04127845
       n    G rho     adelie glmnet  sparsegl   gglasso     grpnet
[1,] 100 1000   0 0.01966667     NA 0.1213333 0.1486667 0.04666667
       n    G rho     adelie glmnet   sparsegl    gglasso     grpnet
[1,] 100 1000   0 0.02344173     NA 0.02343786 0.02343788 0.02343982
       n    G rho     adelie glmnet  sparsegl   gglasso    grpnet
[1,] 100 5000   0 0.03966667     NA 0.4266667 0.5516667 0.1516667
       n    G rho     adelie glmnet   sparsegl    gglasso     grpnet
[1,] 100 5000   0 0.02116939     NA 0.02116681 0.02116663 0.02116824
       n     G rho adelie glmnet  sparsegl  gglasso    grpnet
[1,] 100 10000   0  0.072     NA 0.8733333 1.094333 0.2906667
       n     G rho     adelie glmnet   sparsegl   gglasso     

In [20]:
write.table(out.gaussian$time, "../data/group_lasso_sim_gaussian_time.csv", row.names=FALSE)
write.table(out.gaussian$objective, "../data/group_lasso_sim_gaussian_objective.csv", row.names=FALSE)

In [21]:
out.binomial <- run_benchmarks(configs, generate.data, obj_tol=1e-4, family="binomial")

       n   G rho     adelie glmnet sparsegl   gglasso     grpnet
[1,] 100 100   0 0.02933333     NA    0.109 0.1106667 0.08566667
       n   G rho     adelie glmnet   sparsegl    gglasso     grpnet
[1,] 100 100   0 0.07066608     NA 0.07067291 0.07067276 0.07067923
       n    G rho     adelie glmnet  sparsegl   gglasso grpnet
[1,] 100 1000   0 0.05333333     NA 0.2286667 0.2676667  0.153
       n    G rho     adelie glmnet   sparsegl    gglasso     grpnet
[1,] 100 1000   0 0.04459816     NA 0.04461235 0.04461249 0.04461604
       n    G rho     adelie glmnet  sparsegl   gglasso    grpnet
[1,] 100 5000   0 0.08333333     NA 0.5126667 0.6813333 0.2646667
       n    G rho     adelie glmnet   sparsegl    gglasso     grpnet
[1,] 100 5000   0 0.04624429     NA 0.04626761 0.04626692 0.04626955
       n     G rho    adelie glmnet sparsegl  gglasso    grpnet
[1,] 100 10000   0 0.1333333     NA    0.856 1.146333 0.3623333
       n     G rho    adelie glmnet   sparsegl    gglasso     grpnet
[1,

Warning message in run_benchmarks(configs, generate.data, obj_tol = 1e-04, family = "binomial"):
"n=100, G=5000, rho=0.95 pkg=grpnet"


       n    G  rho    adelie glmnet  sparsegl  gglasso    grpnet
[1,] 100 5000 0.95 0.1303333     NA 0.9216667 1.170667 0.5363333
       n    G  rho    adelie glmnet  sparsegl   gglasso    grpnet
[1,] 100 5000 0.95 0.1044078     NA 0.1044618 0.1044648 0.1045501
       n     G  rho adelie glmnet sparsegl  gglasso    grpnet
[1,] 100 10000 0.95  0.226     NA     1.16 1.611333 0.6373333
       n     G  rho    adelie glmnet  sparsegl   gglasso    grpnet
[1,] 100 10000 0.95 0.1196849     NA 0.1197773 0.1197729 0.1197744
       n     G  rho adelie glmnet sparsegl  gglasso   grpnet
[1,] 100 50000 0.95  0.399     NA 4.570333 5.752667 1.782333
       n     G  rho     adelie glmnet sparsegl    gglasso    grpnet
[1,] 100 50000 0.95 0.08791157     NA 0.087959 0.08796616 0.0879771
       n     G  rho adelie glmnet sparsegl  gglasso   grpnet
[1,] 100 1e+05 0.95  1.233     NA 8.404667 11.75067 3.243333
       n     G  rho    adelie glmnet  sparsegl   gglasso   grpnet
[1,] 100 1e+05 0.95 0.1274893     

In [23]:
write.table(out.binomial$time, "../data/group_lasso_sim_binomial_time.csv", row.names=FALSE)
write.table(out.binomial$objective, "../data/group_lasso_sim_binomial_objective.csv", row.names=FALSE)

In [25]:
data <- generate.data(1000, 10000, 0.95, family="binomial")
X <- data$X
y <- data$y
write.table(X, "../data/test_X_binomial_1000_10000_095.csv", row.names=FALSE, col.names=FALSE)
write.table(y, "../data/test_y_binomial_1000_10000_095.csv", row.names=FALSE, col.names=FALSE)

## Real Data

In [8]:
out <- c()

In [10]:
# prostate
require(sda)
data(singh2002)

In [11]:
X = singh2002$x
y = as.numeric(singh2002$y == "cancer")
X = t(t(X) - colMeans(X))
X = t(t(X) / sqrt(colMeans(X**2)))
n = nrow(X)
G = ncol(X)
X <- sapply(1:G, function(i) cbind(X[,i], X[,i]**2, X[,i]**3))
X <- matrix(X, n, 3 * G)
X = t(t(X) - colMeans(X))
X = t(t(X) / sqrt(colMeans(X**2)))
dim(X)

[1]   102 18099

In [12]:
n_sims <- 3
bench.adelie <- benchmark(X, y, G, NA, "adelie", n_sims, family="binomial")
bench.sparsegl <- benchmark(X, y, G, bench.adelie$out$lmdas, "sparsegl", n_sims, family="binomial")
bench.gglasso <- benchmark(X, y, G, bench.adelie$out$lmdas, "gglasso", n_sims, family="binomial")
bench.grpnet <- benchmark(X, y, G, bench.adelie$out$lmdas, "grpnet", n_sims, family="binomial")

In [13]:
out <- rbind(
    out,
    c(
        "name"="prostate",
        "n"=nrow(X),
        "G"=G,
        "p"=ncol(X),
        "adelie"=bench.adelie$time,
        "sparsegl"=bench.sparsegl$time,
        "gglasso"=bench.gglasso$time,
        "grpnet"=bench.grpnet$time
    )
)

In [14]:
# UJ Indoor Location
df <- read.table("../data/ujindoorloc.csv", sep=",", header=TRUE)

In [15]:
X = df[, 1:520]
y = df[, "LONGITUDE"]
X = t(t(X) - colMeans(X))
X = t(t(X) / sqrt(colMeans(X**2)))
X = X[, colSums(is.na(X)) == 0]
n = nrow(X)
G = ncol(X)
X <- sapply(1:G, function(i) cbind(X[,i], X[,i]**2, X[,i]**3))
X <- matrix(X, n, 3 * G)
X = t(t(X) - colMeans(X))
X = t(t(X) / sqrt(colMeans(X**2)))
y <- y - mean(y)
y <- y / sqrt(mean(y**2))
dim(X)

[1] 19937  1395

In [16]:
n_sims <- 3
bench.adelie <- benchmark(X, y, G, NA, "adelie", n_sims, family="gaussian")
bench.sparsegl <- benchmark(X, y, G, bench.adelie$out$lmdas, "sparsegl", n_sims, family="gaussian")
bench.gglasso <- benchmark(X, y, G, bench.adelie$out$lmdas, "gglasso", n_sims, family="gaussian")
bench.grpnet <- benchmark(X, y, G, bench.adelie$out$lmdas, "grpnet", n_sims, family="gaussian")

Warning message in render_inputs_(y, weights):
"Normalizing weights to sum to 1."
Warning message in render_inputs_(y, weights):
"Normalizing weights to sum to 1."
Warning message in render_inputs_(y, weights):
"Normalizing weights to sum to 1."


In [17]:
out <- rbind(
    out,
    c(
        "name"="ujiindoorloc",
        "n"=nrow(X),
        "G"=G,
        "p"=ncol(X),
        "adelie"=bench.adelie$time,
        "sparsegl"=bench.sparsegl$time,
        "gglasso"=bench.gglasso$time,
        "grpnet"=bench.grpnet$time
    )
)

In [13]:
# leukemia
df <- read.csv("../data/leukemia.csv")

In [15]:
X = df[, 7:ncol(df)]
X = t(t(X) - colMeans(X))
X = t(t(X) / sqrt(colMeans(X**2)))
n = nrow(X)
G = ncol(X)
y = as.logical(df[,"cancer"] == "allB")
X <- sapply(1:G, function(i) cbind(X[,i], X[,i]**2, X[,i]**3))
X <- matrix(X, n, 3 * G)
X = t(t(X) - colMeans(X))
X = t(t(X) / sqrt(colMeans(X**2)))

In [16]:
n_sims <- 3
bench.adelie <- benchmark(X, y, G, NA, "adelie", n_sims, family="binomial")
bench.sparsegl <- benchmark(X, y, G, bench.adelie$out$lmdas, "sparsegl", n_sims, family="binomial")
bench.gglasso <- benchmark(X, y, G, bench.adelie$out$lmdas, "gglasso", n_sims, family="binomial")
bench.grpnet <- benchmark(X, y, G, bench.adelie$out$lmdas, "grpnet", n_sims, family="binomial")

In [21]:
out <- rbind(
    out,
    c(
        "name"="leukemia",
        "n"=nrow(X),
        "G"=G,
        "p"=ncol(X),
        "adelie"=bench.adelie$time,
        "sparsegl"=bench.sparsegl$time,
        "gglasso"=bench.gglasso$time,
        "grpnet"=bench.grpnet$time
    )
)

In [22]:
write.table(out, "../data/group_lasso_real.csv", row.names=FALSE)

## Benchmark against glmnet 

In [8]:
generate.data.lasso <- function(
    n, G, rho, snr=3, seed=0, family="gaussian"
)
{
    set.seed(seed)
    W <- rnorm(n)
    Z <- matrix(rnorm(n * G), n, G)
    X <- sqrt(1-rho) * Z + sqrt(rho) * W
    beta = (-1) ** (1:G) * exp(-2 * (1:G - 1) / 20) 
    signal = rho * sum(beta) ** 2 + (1-rho) * sum(beta ** 2)
    noise = sqrt(signal / snr)
    eta = X %*% beta + noise * rnorm(n)
    if (family == "gaussian") {
        y <- eta
        # standardize
        y <- y - mean(y)
        y <- y / sqrt(mean(y ** 2))
    } else {
        y <- rbinom(n, 1, 1/(1 + exp(-eta)))
    }

    # standardize
    X <- t(t(X) - colMeans(X))
    X <- t(t(X) / sqrt(colMeans(X ** 2)))

    list(X=X, y=y)
    list(X=X, y=y)
}

In [9]:
out.gaussian.lasso <- run_benchmarks(configs, generate.data.lasso, obj_tol=1e-4, family="gaussian")

       n   G rho      adelie      glmnet   sparsegl gglasso     grpnet
[1,] 100 100   0 0.005333333 0.004666667 0.04066667   0.042 0.02566667
       n   G rho     adelie     glmnet sparsegl gglasso     grpnet
[1,] 100 100   0 0.04408227 0.04408214  0.04407 0.04407 0.04407014
       n    G rho adelie      glmnet sparsegl    gglasso grpnet
[1,] 100 1000   0  0.007 0.005333333    0.067 0.09566667  0.028
       n    G rho     adelie     glmnet   sparsegl    gglasso     grpnet
[1,] 100 1000   0 0.01973832 0.01973922 0.01973275 0.01973248 0.01973317
       n    G rho     adelie     glmnet  sparsegl   gglasso grpnet
[1,] 100 5000   0 0.01633333 0.02266667 0.2456667 0.3403333  0.069
       n    G rho     adelie     glmnet   sparsegl    gglasso     grpnet
[1,] 100 5000   0 0.01704313 0.01704312 0.01704138 0.01704162 0.01704166
       n     G rho     adelie     glmnet sparsegl   gglasso    grpnet
[1,] 100 10000   0 0.02933333 0.05433333    0.433 0.6476667 0.1343333
       n     G rho     adelie 

In [10]:
write.table(out.gaussian.lasso$time, "../data/lasso_sim_gaussian_time.csv", row.names=FALSE)
write.table(out.gaussian.lasso$objective, "../data/lasso_sim_gaussian_objective.csv", row.names=FALSE)

In [59]:
out.binomial.lasso <- run_benchmarks(configs, generate.data.lasso, obj_tol=1e-4, family="binomial")

       n   G rho adelie glmnet   sparsegl   gglasso grpnet
[1,] 100 100   0  0.008  0.005 0.09533333 0.1023333  0.139
       n   G rho     adelie     glmnet   sparsegl    gglasso     grpnet
[1,] 100 100   0 0.07422602 0.07422613 0.07424536 0.07424342 0.07423458
       n    G rho      adelie      glmnet sparsegl   gglasso grpnet
[1,] 100 1000   0 0.008666667 0.007666667    0.077 0.1113333  0.089
       n    G rho     adelie     glmnet   sparsegl    gglasso     grpnet
[1,] 100 1000   0 0.04861151 0.04861158 0.04862541 0.04862459 0.04861449
       n    G rho     adelie glmnet sparsegl gglasso    grpnet
[1,] 100 5000   0 0.01866667   0.02    0.232   0.389 0.1373333
       n    G rho     adelie     glmnet   sparsegl   gglasso     grpnet
[1,] 100 5000   0 0.04669334 0.04669359 0.04670215 0.0467017 0.04669791
       n     G rho     adelie     glmnet  sparsegl   gglasso    grpnet
[1,] 100 10000   0 0.03333333 0.03633333 0.3973333 0.7126667 0.1746667
       n     G rho     adelie     glmnet   s

Warning message in run_benchmarks(configs, generate.data.lasso, obj_tol = 1e-04, :
"n=1000, G=500, rho=0.95 pkg=sparsegl"


        n   G  rho    adelie    glmnet sparsegl  gglasso   grpnet
[1,] 1000 500 0.95 0.2693333 0.2983333 2.654667 3.461667 7.036333
        n   G  rho    adelie    glmnet  sparsegl   gglasso    grpnet
[1,] 1000 500 0.95 0.5263886 0.5263682 0.5267852 0.5263736 0.5264437


Warning message in run_benchmarks(configs, generate.data.lasso, obj_tol = 1e-04, :
"n=1000, G=1000, rho=0.95 pkg=sparsegl"


        n    G  rho    adelie glmnet sparsegl  gglasso   grpnet
[1,] 1000 1000 0.95 0.5463333  0.494 4.743333 5.755667 13.79367
        n    G  rho    adelie    glmnet  sparsegl   gglasso    grpnet
[1,] 1000 1000 0.95 0.4255891 0.4255824 0.4261212 0.4256337 0.4256104


Warning message in run_benchmarks(configs, generate.data.lasso, obj_tol = 1e-04, :
"n=1000, G=5000, rho=0.95 pkg=sparsegl"


        n    G  rho   adelie   glmnet sparsegl  gglasso grpnet
[1,] 1000 5000 0.95 1.604667 1.351333    6.367 16.47633  17.67
        n    G  rho    adelie    glmnet  sparsegl   gglasso    grpnet
[1,] 1000 5000 0.95 0.3129694 0.3129166 0.3132859 0.3129687 0.3129503


Warning message in run_benchmarks(configs, generate.data.lasso, obj_tol = 1e-04, :
"n=1000, G=10000, rho=0.95 pkg=sparsegl"


        n     G  rho   adelie   glmnet sparsegl  gglasso   grpnet
[1,] 1000 10000 0.95 1.309333 1.159333    8.159 12.45567 18.55233
        n     G  rho   adelie    glmnet  sparsegl   gglasso    grpnet
[1,] 1000 10000 0.95 0.234033 0.2339613 0.2342282 0.2340418 0.2340089


In [61]:
write.table(out.binomial.lasso$time, "../data/lasso_sim_binomial_time.csv", row.names=FALSE)
write.table(out.binomial.lasso$objective, "../data/lasso_sim_binomial_objective.csv", row.names=FALSE)

## Real Data Lasso

In [27]:
out <- c()

In [28]:
# prostate
library(sda)
data(singh2002)

In [29]:
X = singh2002$x
y = as.numeric(singh2002$y == "cancer")
X = t(t(X) - colMeans(X))
X = t(t(X) / sqrt(colMeans(X**2)))
n = nrow(X)
G = ncol(X)
dim(X)

[1]  102 6033

In [30]:
n_sims <- 3
bench.adelie <- benchmark(X, y, G, NA, "adelie", n_sims, family="binomial")
bench.glmnet <- benchmark(X, y, G, bench.adelie$out$lmdas, "glmnet", n_sims, family="binomial")
bench.sparsegl <- benchmark(X, y, G, bench.adelie$out$lmdas, "sparsegl", n_sims, family="binomial")
bench.gglasso <- benchmark(X, y, G, bench.adelie$out$lmdas, "gglasso", n_sims, family="binomial")
bench.grpnet <- benchmark(X, y, G, bench.adelie$out$lmdas, "grpnet", n_sims, family="binomial")

In [31]:
out <- rbind(
    out,
    c(
        "name"="prostate",
        "n"=nrow(X),
        "G"=G,
        "p"=ncol(X),
        "adelie"=bench.adelie$time,
        "glmnet"=bench.glmnet$time,
        "sparsegl"=bench.sparsegl$time,
        "gglasso"=bench.gglasso$time,
        "grpnet"=bench.grpnet$time
    )
)

In [32]:
# UJ Indoor Location
df <- read.table("../data/ujindoorloc.csv", sep=",", header=TRUE)

In [33]:
X = df[, 1:520]
y = df[, "LONGITUDE"]
X = t(t(X) - colMeans(X))
X = t(t(X) / sqrt(colMeans(X**2)))
X = X[, colSums(is.na(X)) == 0]
n = nrow(X)
G = ncol(X)
y <- y - mean(y)
y <- y / sqrt(mean(y**2))
dim(X)

[1] 19937   465

In [34]:
n_sims <- 3
bench.adelie <- benchmark(X, y, G, NA, "adelie", n_sims, family="gaussian")
bench.glmnet <- benchmark(X, y, G, bench.adelie$out$lmdas, "glmnet", n_sims, family="gaussian")
bench.sparsegl <- benchmark(X, y, G, bench.adelie$out$lmdas, "sparsegl", n_sims, family="gaussian")
bench.gglasso <- benchmark(X, y, G, bench.adelie$out$lmdas, "gglasso", n_sims, family="gaussian")
bench.grpnet <- benchmark(X, y, G, bench.adelie$out$lmdas, "grpnet", n_sims, family="gaussian")

Warning message in render_inputs_(y, weights):
"Normalizing weights to sum to 1."


Warning message in render_inputs_(y, weights):
"Normalizing weights to sum to 1."
Warning message in render_inputs_(y, weights):
"Normalizing weights to sum to 1."


In [35]:
out <- rbind(
    out,
    c(
        "name"="ujiindoorloc",
        "n"=nrow(X),
        "G"=G,
        "p"=ncol(X),
        "adelie"=bench.adelie$time,
        "glmnet"=bench.glmnet$time,
        "sparsegl"=bench.sparsegl$time,
        "gglasso"=bench.gglasso$time,
        "grpnet"=bench.grpnet$time
    )
)

In [36]:
# leukemia
df <- read.csv("../data/leukemia.csv")

In [37]:
X = df[, 7:ncol(df)]
X = t(t(X) - colMeans(X))
X = t(t(X) / sqrt(colMeans(X**2)))
n = nrow(X)
G = ncol(X)
y = as.logical(df[,"cancer"] == "allB")

In [38]:
n_sims <- 3
bench.adelie <- benchmark(X, y, G, NA, "adelie", n_sims, family="binomial")
bench.glmnet <- benchmark(X, y, G, bench.adelie$out$lmdas, "glmnet", n_sims, family="binomial")
bench.sparsegl <- benchmark(X, y, G, bench.adelie$out$lmdas, "sparsegl", n_sims, family="binomial")
bench.gglasso <- benchmark(X, y, G, bench.adelie$out$lmdas, "gglasso", n_sims, family="binomial")
bench.grpnet <- benchmark(X, y, G, bench.adelie$out$lmdas, "grpnet", n_sims, family="binomial")

In [39]:
out <- rbind(
    out,
    c(
        "name"="leukemia",
        "n"=nrow(X),
        "G"=G,
        "p"=ncol(X),
        "adelie"=bench.adelie$time,
        "glmnet"=bench.glmnet$time,
        "sparsegl"=bench.sparsegl$time,
        "gglasso"=bench.gglasso$time,
        "grpnet"=bench.grpnet$time
    )
)

In [40]:
write.table(out, "../data/lasso_real.csv", row.names=FALSE)